In [2]:
import numpy as np
import numpy.linalg as lin
import scipy.stats as sts
import scipy.integrate as intgr
import scipy.optimize as opt
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
cmap1 = matplotlib.cm.get_cmap('summer')

In [218]:
# Define the true parameters
a_true = 10
b_true = 5
n = 100000
np.random.seed(42)

def model(x, a, b):
    return a + b*x

# Define the function that generates the data
# be careful on noise strengths compared to signal!!!!
def generate_data(a, b, n):
    x = np.random.normal(0, 1, size=n)
    y = model(x, a, b) + np.random.normal(0, 0.1, size=n)
    return x, y

# Generate the data
x, y = generate_data(a_true, b_true, n)

# Instruments 
z1 = x + np.random.normal(0, 1, size=n)
z2 = x + np.random.normal(0, 1, size=n)
z = np.c_[z1, z2]

In [219]:
def g_vec(y, x, z, params):
    a, b = params
    error = y - model(x, a, b)
    m1 = np.mean(z[:, 0] * error)
    m2 = np.mean(z[:, 1] * error)
    g =  np.column_stack((m1, m2)).T
    return g

In [220]:
def criterion(params, *args):
    a, b = params
    y, x, z, W = args
    g = g_vec(y, x, z, params)
    crit_val = np.dot(np.dot(g.T, W), g) 
    # crit_val = np.dot(g.T, g) 
    return crit_val

In [221]:
a_init = 1
b_init = 1
params_init = np.array([a_init, b_init])
W_hat = np.eye(2)
gmm_args = (y, x, z, W_hat)
criterion([a_init, b_init], *gmm_args)

array([[2.01593132]])

In [222]:
a_init = 1
b_init = 1
params_init = np.array([a_init, b_init])
W_hat = np.eye(2)
gmm_args = (y, x, z, W_hat)
results = opt.minimize(criterion, params_init, args=(gmm_args), tol=1e-6, options  = {"maxiter":100000, "disp":True})#, method='SLSQP')
results

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 12
         Function evaluations: 60
         Gradient evaluations: 20


      fun: 2.2455430164912666e-15
 hess_inv: array([[ 5.24302480e+05, -8.06118229e+01],
       [-8.06118229e+01,  2.54014075e-01]])
      jac: array([5.75579259e-11, 1.51749736e-07])
  message: 'Optimization terminated successfully.'
     nfev: 60
      nit: 12
     njev: 20
   status: 0
  success: True
        x: array([9.99028744, 5.00024418])

In [223]:
results.x

array([9.99028744, 5.00024418])